In [6]:
import numpy as np
import csv
import random

In [7]:
male_data = []
with open('MALE.csv') as csvfile:
    csv_reader = csv.reader(csvfile)  
    male_header = next(csv_reader)  
    for row in csv_reader: 
        male_data.append(row)
        
female_data = []
with open('FEMALE.csv') as csvfile:
    csv_reader = csv.reader(csvfile)  
    female_header = next(csv_reader)  
    for row in csv_reader: 
        female_data.append(row)
        
mixed_data = []
with open('MIXED.csv') as csvfile:
    csv_reader = csv.reader(csvfile)  
    mixed_header = next(csv_reader)  
    for row in csv_reader: 
        mixed_data.append(row)
        
male_data = [[int(a) for a in row] for row in male_data]
female_data = [[int(b) for b in row] for row in female_data]
mixed_data = [[int(c) for c in row] for row in mixed_data]

In [8]:
# shuffle dataset
random.shuffle(male_data)
random.shuffle(female_data)
random.shuffle(mixed_data)

In [9]:
print(female_data[1])

[1, 26, 26, 3, 1, 1, 26]


In [10]:
# get score(label)
male_score = []
for e in male_data:
    male_score.append(e[6])
    del e[6]
    
female_score = []
for e in female_data:
    female_score.append(e[6])
    del e[6]
    
mixed_score = []
for e in mixed_data:
    mixed_score.append(e[6])
    del e[6]

In [11]:
# divide train, dev and test set
test_male_data_x = male_data[:100]
dev_male_data_x = male_data[100:200]
train_male_data_x = male_data[200:]

test_male_data_y = male_score[:100]
dev_male_data_y = male_score[100:200]
train_male_data_y = male_score[200:]

test_female_data_x = female_data[:100]
dev_female_data_x = female_data[100:200]
train_female_data_x = female_data[200:]

test_female_data_y = female_score[:100]
dev_female_data_y = female_score[100:200]
train_female_data_y = female_score[200:]

test_mixed_data_x = mixed_data[:100]
dev_mixed_data_x = mixed_data[100:200]
train_mixed_data_x = mixed_data[200:]

test_mixed_data_y = mixed_score[:100]
dev_mixed_data_y = mixed_score[100:200]
train_mixed_data_y = mixed_score[200:]

In [12]:
print(len(train_mixed_data_x),len(train_mixed_data_y))

7089 7089


In [13]:
# neural network
from sklearn.neural_network import MLPRegressor
from sklearn import linear_model
from sklearn.metrics import mean_squared_error

In [24]:
# mlp regression
def mlp_regression(x_train, y_train, x_test):
    regr = MLPRegressor(max_iter=800).fit(x_train, y_train)

    x_test = np.array(x_test)
    y_pred = regr.predict(x_test)
    
    return y_pred

In [15]:
# linear regression
def lr_regression(x_train, y_train, x_test):
    regr = linear_model.LinearRegression().fit(x_train, y_train)

    x_test = np.array(x_test)
    y_pred = regr.predict(x_test)
    
    return y_pred

In [16]:
# linint mlp
def get_pred_mlp(src_x_train, src_y_train, tgt_x_train, tgt_y_train, x_test):    
    y1_pred = mlp_regression(src_x_train, src_y_train, x_test)
    y2_pred = mlp_regression(tgt_x_train, tgt_y_train, x_test)
    
    return y1_pred, y2_pred

In [17]:
# linint lr
def get_pred_lr(src_x_train, src_y_train, tgt_x_train, tgt_y_train, x_test):
    y1_pred = lr_regression(src_x_train, src_y_train, x_test)
    y2_pred = lr_regression(tgt_x_train, tgt_y_train, x_test)
    
    return y1_pred, y2_pred

In [25]:
# male srconly
src_x_train = train_female_data_x + train_mixed_data_x
src_x_train = np.array(src_x_train)
src_y_train = train_female_data_y + train_mixed_data_y
src_y_train = np.array(src_y_train)
y_test = np.array(test_male_data_y)

mlp_y_pred = mlp_regression(src_x_train, src_y_train, test_male_data_x)
lr_y_pred = lr_regression(src_x_train, src_y_train, test_male_data_x)

print(mean_squared_error(y_test, mlp_y_pred), mean_squared_error(y_test, lr_y_pred))

131.37658703682314 164.39943663546615


In [26]:
# female srconly
src_x_train = train_male_data_x + train_mixed_data_x
src_x_train = np.array(src_x_train)
src_y_train = train_male_data_y + train_mixed_data_y
src_y_train = np.array(src_y_train)

y_test = np.array(test_female_data_y)

mlp_y_pred = mlp_regression(src_x_train, src_y_train, test_female_data_x)
lr_y_pred = lr_regression(src_x_train, src_y_train, test_female_data_x)

print(mean_squared_error(y_test, mlp_y_pred), mean_squared_error(y_test, lr_y_pred))

120.12673556379139 175.4876802912998


In [27]:
# mixed srconly
src_x_train = train_male_data_x + train_female_data_x
src_x_train = np.array(src_x_train)
src_y_train = train_male_data_y + train_female_data_y
src_y_train = np.array(src_y_train)

y_test = np.array(test_mixed_data_y)

mlp_y_pred = mlp_regression(src_x_train, src_y_train, test_mixed_data_x)
lr_y_pred = lr_regression(src_x_train, src_y_train, test_mixed_data_x)

print(mean_squared_error(y_test, mlp_y_pred), mean_squared_error(y_test, lr_y_pred))

140.8210621639053 173.3934186660481


In [28]:
# male tgtonly
tgt_x_train = np.array(train_male_data_x)
tgt_y_train = np.array(train_male_data_y)

y_test = np.array(test_male_data_y)

mlp_y_pred = mlp_regression(tgt_x_train, tgt_y_train, test_male_data_x)
lr_y_pred = lr_regression(tgt_x_train, tgt_y_train, test_male_data_x)

print(mean_squared_error(y_test, mlp_y_pred), mean_squared_error(y_test, lr_y_pred))

135.8755717550831 159.90091162701756


In [29]:
# female tgtonly
tgt_x_train = np.array(train_female_data_x)
tgt_y_train = np.array(train_female_data_y)

y_test = np.array(test_female_data_y)

mlp_y_pred = mlp_regression(tgt_x_train, tgt_y_train, test_female_data_x)
lr_y_pred = lr_regression(tgt_x_train, tgt_y_train, test_female_data_x)

print(mean_squared_error(y_test, mlp_y_pred), mean_squared_error(y_test, lr_y_pred))

153.51629384171895 156.94152882532816


In [30]:
# mixed tgtonly
tgt_x_train = np.array(train_mixed_data_x)
tgt_y_train = np.array(train_mixed_data_y)

y_test = np.array(test_mixed_data_y)

mlp_y_pred = mlp_regression(tgt_x_train, tgt_y_train, test_mixed_data_x)
lr_y_pred = lr_regression(tgt_x_train, tgt_y_train, test_mixed_data_x)

print(mean_squared_error(y_test, mlp_y_pred), mean_squared_error(y_test, lr_y_pred))

125.0393228162836 173.44071670380296


In [31]:
# male all
all_x_train = train_male_data_x[:100] + train_female_data_x + train_mixed_data_x
all_x_train = np.array(all_x_train)
all_y_train = train_male_data_y[:100] + train_female_data_y + train_mixed_data_y
all_y_train = np.array(all_y_train)

y_test = np.array(test_male_data_y)

mlp_y_pred = mlp_regression(all_x_train, all_y_train, test_male_data_x)
lr_y_pred = lr_regression(all_x_train, all_y_train, test_male_data_x)

print(mean_squared_error(y_test, mlp_y_pred), mean_squared_error(y_test, lr_y_pred))

134.51452108210168 164.34584353616444


In [32]:
# female all
all_x_train = train_female_data_x[:100] + train_male_data_x + train_mixed_data_x
all_x_train = np.array(all_x_train)
all_y_train = train_female_data_y[:100] + train_male_data_y + train_mixed_data_y
all_y_train = np.array(all_y_train)

y_test = np.array(test_female_data_y)

mlp_y_pred = mlp_regression(all_x_train, all_y_train, test_female_data_x)
lr_y_pred = lr_regression(all_x_train, all_y_train, test_female_data_x)

print(mean_squared_error(y_test, mlp_y_pred), mean_squared_error(y_test, lr_y_pred))

119.9039406525048 175.24142249722206


In [33]:
# mixed all
all_x_train = train_mixed_data_x[:100] + train_male_data_x + train_female_data_x
all_x_train = np.array(all_x_train)
all_y_train = train_mixed_data_y[:100] + train_male_data_y + train_female_data_y
all_y_train = np.array(all_y_train)

y_test = np.array(test_mixed_data_y)

mlp_y_pred = mlp_regression(all_x_train, all_y_train, test_mixed_data_x)
lr_y_pred = lr_regression(all_x_train, all_y_train, test_mixed_data_x)

print(mean_squared_error(y_test, mlp_y_pred), mean_squared_error(y_test, lr_y_pred))

140.27667636011248 173.42552803875557


In [34]:
# male weighted
all_x_train = train_male_data_x[:100] * 10 + train_female_data_x + train_mixed_data_x
all_x_train = np.array(all_x_train)
all_y_train = train_male_data_y[:100] * 10 + train_female_data_y + train_mixed_data_y
all_y_train = np.array(all_y_train)

y_test = np.array(test_male_data_y)

mlp_y_pred = mlp_regression(all_x_train, all_y_train, test_male_data_x)
lr_y_pred = lr_regression(all_x_train, all_y_train, test_male_data_x)

print(mean_squared_error(y_test, mlp_y_pred), mean_squared_error(y_test, lr_y_pred))

136.54962338912324 164.01902271705302


In [35]:
# female weighted
all_x_train = train_female_data_x[:100] * 10 + train_male_data_x + train_mixed_data_x
all_x_train = np.array(all_x_train)
all_y_train = train_female_data_y[:100] * 10 + train_male_data_y + train_mixed_data_y
all_y_train = np.array(all_y_train)

y_test = np.array(test_female_data_y)

mlp_y_pred = mlp_regression(all_x_train, all_y_train, test_female_data_x)
lr_y_pred = lr_regression(all_x_train, all_y_train, test_female_data_x)

print(mean_squared_error(y_test, mlp_y_pred), mean_squared_error(y_test, lr_y_pred))

124.12577044486768 173.46792437109863


In [36]:
# mixed weighted
all_x_train = train_mixed_data_x[:100] * 10 + train_male_data_x + train_female_data_x
all_x_train = np.array(all_x_train)
all_y_train = train_mixed_data_y[:100] * 10 + train_male_data_y + train_female_data_y
all_y_train = np.array(all_y_train)

y_test = np.array(test_mixed_data_y)

mlp_y_pred = mlp_regression(all_x_train, all_y_train, test_mixed_data_x)
lr_y_pred = lr_regression(all_x_train, all_y_train, test_mixed_data_x)

print(mean_squared_error(y_test, mlp_y_pred), mean_squared_error(y_test, lr_y_pred))

139.69158638194804 173.77365796537288


In [37]:
# pred mlp
def pred_mlp(src_x_train, src_y_train, tgt_x_train, tgt_y_train, x_test, y_test):
    regr = MLPRegressor(max_iter=800).fit(src_x_train, src_y_train)
    regr = MLPRegressor(max_iter=800).fit(tgt_x_train, tgt_y_train)

    x_test = np.array(x_test)
    y_test = np.array(y_test)
    y_pred = regr.predict(x_test)

    return mean_squared_error(y_test, y_pred)

In [38]:
# pred lr
def pred_lr(src_x_train, src_y_train, tgt_x_train, tgt_y_train, x_test, y_test):
    regr = linear_model.LinearRegression().fit(src_x_train, src_y_train)
    regr = linear_model.LinearRegression().fit(tgt_x_train, tgt_y_train)

    x_test = np.array(x_test)
    y_test = np.array(y_test)
    y_pred = regr.predict(x_test)

    return mean_squared_error(y_test, y_pred)

In [39]:
# male pred
src_x_train = train_female_data_x + train_mixed_data_x
src_x_train = np.array(src_x_train)
src_y_train = train_female_data_y + train_mixed_data_y
src_y_train = np.array(src_y_train)
tgt_x_train = np.array(train_male_data_x)
tgt_y_train = np.array(train_male_data_y)

mlp_mse = pred_mlp(src_x_train, src_y_train, tgt_x_train, tgt_y_train, test_male_data_x, test_male_data_y)
lr_mse = pred_lr(src_x_train, src_y_train, tgt_x_train, tgt_y_train, test_male_data_x, test_male_data_y)

print(mlp_mse, lr_mse)

135.93192300016352 159.90091162701756


In [40]:
# female pred
src_x_train = train_male_data_x + train_mixed_data_x
src_x_train = np.array(src_x_train)
src_y_train = train_male_data_y + train_mixed_data_y
src_y_train = np.array(src_y_train)
tgt_x_train = np.array(train_female_data_x)
tgt_y_train = np.array(train_female_data_y)

mlp_mse = pred_mlp(src_x_train, src_y_train, tgt_x_train, tgt_y_train, test_female_data_x, test_female_data_y)
lr_mse = pred_lr(src_x_train, src_y_train, tgt_x_train, tgt_y_train, test_female_data_x, test_female_data_y)

print(mlp_mse, lr_mse)

117.2575801593722 156.94152882532816


In [41]:
# mixed pred
src_x_train = train_male_data_x + train_female_data_x
src_x_train = np.array(src_x_train)
src_y_train = train_male_data_y + train_female_data_y
src_y_train = np.array(src_y_train)
tgt_x_train = np.array(train_mixed_data_x)
tgt_y_train = np.array(train_mixed_data_y)

mlp_mse = pred_mlp(src_x_train, src_y_train, tgt_x_train, tgt_y_train, test_mixed_data_x, test_mixed_data_y)
lr_mse = pred_lr(src_x_train, src_y_train, tgt_x_train, tgt_y_train, test_mixed_data_x, test_mixed_data_y)

print(mlp_mse, lr_mse)

126.08799392568206 173.44071670380296


In [42]:
np.shape(np.array(test_male_data_y))

(100,)

In [43]:
# get value w
def get_mse(y1_pred, y2_pred, y_test, W):   
    best_w = 0
    best_mse = float("inf")
    for w in W:
        y_pred = []
        for i, j in zip(y1_pred, y2_pred):
            summ = w * float(j) + (1-w) * float(i)
            y_pred.append(summ)
        mse = mean_squared_error(y_test, y_pred)
        if (mse < best_mse):
            best_mse = mse
            best_w = w
    
    if best_w == 0 :
        W2 = [best_w, best_w+0.05, best_w+0.1, best_w+0.15, best_w+0.2]
    elif best_w == 1:
        W2 = [best_w-0.2, best_w-0.15, best_w-0.1, best_w-0.05, best_w]
    else:
        W2 = [best_w-0.1, best_w-0.05, best_w, best_w+0.05, best_w+0.1]
        
    for e in W:
        y_pred = []
        for i, j in zip(y1_pred, y2_pred):
            summ = e * float(j) + (1-e) * float(i)
            y_pred.append(summ)
        mse = mean_squared_error(y_test, y_pred)
        if (mse < best_mse):
            best_mse = mse
            best_w = e

    return best_mse    

In [44]:
# male linint
src_x_train = train_female_data_x + train_mixed_data_x
src_x_train = np.array(src_x_train)
src_y_train = train_female_data_y + train_mixed_data_y
src_y_train = np.array(src_y_train)

tgt_x_train = np.array(train_male_data_x)
tgt_y_train = np.array(train_male_data_y)

y_test = np.array(test_male_data_y)

mlp_y1_pred, mlp_y2_pred = get_pred_mlp(src_x_train, src_y_train, tgt_x_train, tgt_y_train,test_male_data_x)
lr_y1_pred, lr_y2_pred = get_pred_lr(src_x_train, src_y_train, tgt_x_train, tgt_y_train, test_male_data_x)

W = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]

mlp_mse = get_mse(mlp_y1_pred, mlp_y2_pred, y_test, W)
lr_mse = get_mse(lr_y1_pred, lr_y2_pred, y_test, W)

print(mlp_mse, lr_mse)

131.077438082092 159.90091162701756


In [45]:
# female linint
src_x_train = train_male_data_x + train_mixed_data_x
src_x_train = np.array(src_x_train)
src_y_train = train_male_data_y + train_mixed_data_y
src_y_train = np.array(src_y_train)

tgt_x_train = np.array(train_female_data_x)
tgt_y_train = np.array(train_female_data_y)

y_test = np.array(test_female_data_y)

mlp_y1_pred, mlp_y2_pred = get_pred_mlp(src_x_train, src_y_train, tgt_x_train, tgt_y_train,test_female_data_x)
lr_y1_pred, lr_y2_pred = get_pred_lr(src_x_train, src_y_train, tgt_x_train, tgt_y_train, test_female_data_x)

W = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]

mlp_mse = get_mse(mlp_y1_pred, mlp_y2_pred, y_test, W)
lr_mse = get_mse(lr_y1_pred, lr_y2_pred, y_test, W)

print(mlp_mse, lr_mse)

121.90878398191047 156.94152882532816


In [46]:
# mixed linint
src_x_train = train_male_data_x + train_female_data_x
src_x_train = np.array(src_x_train)
src_y_train = train_male_data_y + train_female_data_y
src_y_train = np.array(src_y_train)

tgt_x_train = np.array(train_mixed_data_x)
tgt_y_train = np.array(train_mixed_data_y)

y_test = np.array(test_mixed_data_y)

mlp_y1_pred, mlp_y2_pred = get_pred_mlp(src_x_train, src_y_train, tgt_x_train, tgt_y_train,test_mixed_data_x)
lr_y1_pred, lr_y2_pred = get_pred_lr(src_x_train, src_y_train, tgt_x_train, tgt_y_train, test_mixed_data_x)

W = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]

mlp_mse = get_mse(mlp_y1_pred, mlp_y2_pred, y_test, W)
lr_mse = get_mse(lr_y1_pred, lr_y2_pred, y_test, W)

print(mlp_mse, lr_mse)

129.01626984026814 172.88479251098738
